CV file Read

In [59]:
import pandas as pd
import re

# Read in the CSV file 
applicants_2023 = pd.read_csv('C:/Users/ksultana/Videos/Internship/2023Applicants.csv')
print(applicants_2023.head(100))
print(applicants_2023.info())

    record_id  selection_type age  gender_id underrepresented ethnicity_other  \
0           1               0  20          1                7             NaN   
1           2               0  21          2                7             NaN   
2           3               0  19          2                8             NaN   
3           4               2  21          2                8             NaN   
4           5               0  19          1             5,10             NaN   
..        ...             ...  ..        ...              ...             ...   
95         96               0  20          2                7             NaN   
96         97               1  20          1             8,10             NaN   
97         98               0  21          1                3             NaN   
98         99               0  19          1                3             NaN   
99        100               0  20          1                8             NaN   

   disadvantage  local     

In [60]:
def extract_number_or_decimal(value):
    if isinstance(value, (str, bytes)):
        # Extract numerical values and decimals using regular expression
        match = re.search(r'\d+\.\d+|\d+', value)
        if match:
            return match.group()
    return None

# Drop rows where 'age' column has value '#NUM!'
applicants_2023 = applicants_2023[applicants_2023['age'] != '#NUM!']

# Drop rows where 'grade_average' column has value 'N/a'
applicants_2023 = applicants_2023[applicants_2023['grade_average'] != 'N/a']

# Extract numerical values or decimals from 'grade_average' column
applicants_2023['grade_average'] = applicants_2023['grade_average'].apply(extract_number_or_decimal)

# Convert to float
applicants_2023['grade_average'] = applicants_2023['grade_average'].astype(float)

# Drop rows with empty values in 'grade_average' column
applicants_2023 = applicants_2023.dropna(subset=['grade_average'])

# Fill empty values in 'local' column with 0
applicants_2023['local'] = applicants_2023['local'].fillna(0)

# Drop the 'ethnicity_other' column
applicants_2023 = applicants_2023.drop(columns=['ethnicity_other'])

# Drop the 'sigsub' column
applicants_2023 = applicants_2023.drop(columns=['sigsub'])

# Drop the 'application_submission_form_complete' column
applicants_2023 = applicants_2023.drop(columns=['application_submission_form_complete'])

# Drop rows with missing values only for the 'citizenship_2' column
applicants_2023.dropna(subset=['citizenship_2'], inplace=True)

# Drop rows with missing values only for the 'project_one_5' column
applicants_2023.dropna(subset=['project_one_5'], inplace=True)

print(applicants_2023.head(100))

     record_id  selection_type age  gender_id underrepresented disadvantage  \
0            1               0  20          1                7          3,5   
4            5               0  19          1             5,10          3,5   
5            6               0  19          5              3,9            8   
6            7               0  21          1                3            8   
7            8               0  21          2             8,11            8   
..         ...             ...  ..        ...              ...          ...   
106        107               0  21          2                8            8   
107        108               0  18          2                3            8   
108        109               0  18          1                7            8   
109        110               0  19          2                7        3,5,6   
110        111               0  21          1                8            5   

     local           city  state zipcode  ...  conc

In [53]:
# Split the multiple values in the 'underrepresented' column and expand into separate columns
split_columns = applicants_2023['underrepresented'].str.split(',', expand=True)

# Get unique values from all split columns
unique_values = split_columns.unstack().dropna().unique()

# Create new columns for each unique value
for value in unique_values:
    applicants_2023[f'underrepresented_{value}'] = split_columns.apply(lambda row: 1 if str(value) in row.values else 0, axis=1)
    
# Convert specific columns to int
columns_to_convert = ['underrepresented_2', 'underrepresented_3', 'underrepresented_4','underrepresented_5','underrepresented_6',
                     'underrepresented_7','underrepresented_8','underrepresented_9','underrepresented_10', 'underrepresented_11'] 

for column in columns_to_convert:
    applicants_2023[column] = applicants_2023[column].astype(int)

# Print the data types to confirm the changes
print(applicants_2023.dtypes)

# Drop the original 'underrepresented' column
applicants_2023 = applicants_2023.drop(columns=['underrepresented'])

print(applicants_2023.head(100))

record_id                   int64
selection_type              int64
age                        object
gender_id                   int64
underrepresented           object
disadvantage               object
local                     float64
city                       object
state                       int64
zipcode                    object
citizenship                 int64
citizenship_2             float64
covid_question              int64
institution                object
classification              int64
graduation_date            object
undergrad_major            object
minor                      object
grade_average             float64
research_opportunities      int64
available                   int64
summer_classes              int64
concentration_choice1       int64
concentration_choice2       int64
concentration_choice3       int64
research_experience         int64
fp_choice_1                 int64
fp_choice_2                 int64
fp_choice_3                 int64
project_one   

In [54]:
# Split the multiple values in the 'underrepresented' column and expand into separate columns
split_columns = applicants_2023['disadvantage'].str.split(',', expand=True)

# Get unique values from all split columns
unique_values = split_columns.unstack().dropna().unique()

# Create new columns for each unique value
for value in unique_values:
    applicants_2023[f'disadvantage_{value}'] = split_columns.apply(lambda row: 1 if str(value) in row.values else 0, axis=1)
    
# Convert specific columns to int
columns_to_convert = ['disadvantage_1', 'disadvantage_2', 'disadvantage_3','disadvantage_4','disadvantage_5',
                     'disadvantage_6','disadvantage_7','disadvantage_8'] 

for column in columns_to_convert:
    applicants_2023[column] = applicants_2023[column].astype(int)

# Print the data types to confirm the changes
print(applicants_2023.dtypes)

# Drop the original 'underrepresented' column
applicants_2023 = applicants_2023.drop(columns=['disadvantage'])

print(applicants_2023.head(100))

record_id                   int64
selection_type              int64
age                        object
gender_id                   int64
disadvantage               object
local                     float64
city                       object
state                       int64
zipcode                    object
citizenship                 int64
citizenship_2             float64
covid_question              int64
institution                object
classification              int64
graduation_date            object
undergrad_major            object
minor                      object
grade_average             float64
research_opportunities      int64
available                   int64
summer_classes              int64
concentration_choice1       int64
concentration_choice2       int64
concentration_choice3       int64
research_experience         int64
fp_choice_1                 int64
fp_choice_2                 int64
fp_choice_3                 int64
project_one               float64
project_one_2 

In [55]:
# Split the multiple values in the 'underrepresented' column and expand into separate columns
split_columns = applicants_2023['undergrad_major'].str.split(',', expand=True)

# Get unique values from all split columns
unique_values = split_columns.unstack().dropna().unique()

# Create new columns for each unique value
for value in unique_values:
    applicants_2023[f'undergrad_major_{value}'] = split_columns.apply(lambda row: 1 if str(value) in row.values else 0, axis=1)

# Convert specific columns to int
columns_to_convert = ['undergrad_major_1', 'undergrad_major_2', 'undergrad_major_3','undergrad_major_4','undergrad_major_5',
                     'undergrad_major_6','undergrad_major_8','undergrad_major_9','undergrad_major_10','undergrad_major_11',
                      'undergrad_major_12','undergrad_major_13'] 

for column in columns_to_convert:
    applicants_2023[column] = applicants_2023[column].astype(int)

# Print the data types to confirm the changes
print(applicants_2023.dtypes)

    
# Drop the original 'underrepresented' column
applicants_2023 = applicants_2023.drop(columns=['undergrad_major'])

print(applicants_2023.head(100))


record_id              int64
selection_type         int64
age                   object
gender_id              int64
local                float64
                      ...   
undergrad_major_7      int64
undergrad_major_9      int32
undergrad_major_4      int32
undergrad_major_5      int32
undergrad_major_6      int32
Length: 63, dtype: object
     record_id  selection_type age  gender_id  local           city  state  \
0            1               0  20          1    0.0  South Orange      31   
4            5               0  19          1    0.0  Lawrenceville     11   
5            6               0  19          5    0.0    Los Angeles      5   
6            7               0  21          1    0.0  Oklahoma City     37   
7            8               0  21          2    1.0    Chapel Hill     34   
..         ...             ...  ..        ...    ...            ...    ...   
106        107               0  21          2    0.0       Nazareth     39   
107        108               0 

In [56]:
# Fill missing values in the 'minor' column with 0 (for no)
applicants_2023['minor'] = applicants_2023['minor'].fillna(0)

# Replace non-missing values with 1 (for yes)
applicants_2023['minor'] = applicants_2023['minor'].apply(lambda x: 1 if x != 0 else 0)

# Rename the column to indicate its meaning
applicants_2023.rename(columns={'minor': 'has_minor'}, inplace=True)

print(applicants_2023.head(100))


     record_id  selection_type age  gender_id  local           city  state  \
0            1               0  20          1    0.0  South Orange      31   
4            5               0  19          1    0.0  Lawrenceville     11   
5            6               0  19          5    0.0    Los Angeles      5   
6            7               0  21          1    0.0  Oklahoma City     37   
7            8               0  21          2    1.0    Chapel Hill     34   
..         ...             ...  ..        ...    ...            ...    ...   
106        107               0  21          2    0.0       Nazareth     39   
107        108               0  18          2    0.0        Suwanee     11   
108        109               0  18          1    0.0     Orangeburg     41   
109        110               0  19          2    0.0        Hampton     47   
110        111               0  21          1    0.0      Iowa City     16   

    zipcode  citizenship  citizenship_2  ...  undergrad_major_2

In [57]:
# Convert specific columns to int
columns_to_convert = ['age', 'local', 'project_one','project_one_2','project_one_3',
                     'project_one_4','project_one_5'] 

for column in columns_to_convert:
    applicants_2023[column] = applicants_2023[column].astype(int)

# Remove non-numeric characters from 'zipcode'
applicants_2023['zipcode'] = applicants_2023['zipcode'].str.replace(r'\D', '', regex=True)

# Convert 'zipcode' column from object to int
applicants_2023['zipcode'] = applicants_2023['zipcode'].astype(int)


# Convert 'citizenship_2' column from float to int
applicants_2023['citizenship_2'] = applicants_2023['citizenship_2'].astype(int)

applicants_2023['graduation_date'] = pd.to_datetime(applicants_2023['graduation_date'], format='%m/%d/%Y', errors='coerce')



# Print the data types to confirm the changes
print(applicants_2023.dtypes)
print(applicants_2023.info())

applicants_2023.to_csv('C:/Users/ksultana/Videos/Internship/2023Applicants_clean.csv', index=False, header=True)

record_id            int64
selection_type       int64
age                  int32
gender_id            int64
local                int32
                     ...  
undergrad_major_7    int64
undergrad_major_9    int32
undergrad_major_4    int32
undergrad_major_5    int32
undergrad_major_6    int32
Length: 62, dtype: object
<class 'pandas.core.frame.DataFrame'>
Int64Index: 556 entries, 0 to 568
Data columns (total 62 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   record_id               556 non-null    int64         
 1   selection_type          556 non-null    int64         
 2   age                     556 non-null    int32         
 3   gender_id               556 non-null    int64         
 4   local                   556 non-null    int32         
 5   city                    556 non-null    object        
 6   state                   556 non-null    int64         
 7   zipcode                 556